# MMM with Mediation — End-to-End Notebook

Run all cells. Handles path issues so imports from `src/` work even if notebook is inside `notebooks/`.

In [2]:
import sys
from pathlib import Path

# ensure we can import from src/
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(ROOT))

from src.constants import *
from src.data_prep import load_and_clean, stage1_design, stage2_design
from src.fe import apply_channel_transforms
from src.utils import train_valid_split_by_weeks
from src.modeling import ts_grid_search, fit_enet
from src.diagnostics import save_actual_vs_pred, save_residual_plots, summarize

REPORTS = ROOT / "reports"
REPORTS.mkdir(exist_ok=True)


## 1) Load dataset

In [3]:
df = load_and_clean(DATA_PATH)
df.head()

,week,facebook_spend,google_spend,tiktok_spend,instagram_spend,snapchat_spend,social_followers,average_price,promotions,emails_send,sms_send,revenue
0,2023-09-17,6030.80,3130.14,2993.22,1841.08,2204.72,0,101.95,0,102684,20098,83124.16
1,2023-09-24,5241.44,2704.00,0.00,0.00,0.00,0,103.86,0,96573,29920,373.02
2,2023-10-01,5893.00,0.00,0.00,0.00,0.00,0,100.38,0,96797,22304,513.01
3,2023-10-08,7167.16,0.00,0.00,0.00,0.00,0,103.14,1,99098,14171,452.78
4,2023-10-15,5360.29,0.00,0.00,3237.15,0.00,0,107.76,1,120754,30207,41441.95


## 2) Stage 1 — Google as Mediator

In [8]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


df1 = stage1_design(df)
X1 = df1.drop(columns=[DATE_COL, Y_COL, GOOGLE_COL], errors='ignore')
y1 = df1[GOOGLE_COL]
param_grid_1 = {'alpha':[0.001,0.01,0.1,0.5], 'l1_ratio':[0.1,0.5,0.9]}
best1, rmse1, _ = ts_grid_search(X1, y1, param_grid_1, n_folds=N_FOLDS, random_state=SEED)
print('Stage1 best:', best1, '| CV RMSE:', rmse1)
m1 = fit_enet(X1, y1, best1, random_state=SEED)
google_pred = m1.predict(X1)
stage1_out = df1[[DATE_COL]].copy(); stage1_out['google_pred']=google_pred; stage1_out['google_resid']=y1.values - google_pred
full = df.merge(stage1_out, on=DATE_COL, how='left')
full.head()

Stage1 best: {'alpha': 0.5, 'l1_ratio': 0.1} | CV RMSE: 1899.6117038536602


,week,facebook_spend,google_spend,tiktok_spend,instagram_spend,snapchat_spend,social_followers,average_price,promotions,emails_send,sms_send,revenue,google_pred,google_resid
0,2023-09-17,6030.80,3130.14,2993.22,1841.08,2204.72,0,101.95,0,102684,20098,83124.16,NaN,NaN
1,2023-09-24,5241.44,2704.00,0.00,0.00,0.00,0,103.86,0,96573,29920,373.02,NaN,NaN
2,2023-10-01,5893.00,0.00,0.00,0.00,0.00,0,100.38,0,96797,22304,513.01,1331.029511,-1331.029511
3,2023-10-08,7167.16,0.00,0.00,0.00,0.00,0,103.14,1,99098,14171,452.78,607.469937,-607.469937
4,2023-10-15,5360.29,0.00,0.00,3237.15,0.00,0,107.76,1,120754,30207,41441.95,962.017544,-962.017544


## 3) Feature Engineering for Stage 2

In [9]:
paid_cols = SOCIAL_COLS + [GOOGLE_COL]
full = apply_channel_transforms(full, cols=paid_cols, alpha=0.5, beta=1.0, prefix='tr_')
df2 = stage2_design(full)
X2 = df2.drop(columns=[DATE_COL, Y_COL], errors='ignore'); y2 = df2[Y_COL]
train2, test2 = train_valid_split_by_weeks(df2, DATE_COL, TEST_SPLIT_WEEKS)
X2_tr, y2_tr = train2.drop(columns=[DATE_COL, Y_COL], errors='ignore'), train2[Y_COL]
X2_te, y2_te = test2.drop(columns=[DATE_COL, Y_COL], errors='ignore'), test2[Y_COL]
X2_tr.shape, X2_te.shape

((74, 36), (26, 36))

## 4) Stage 2 — Train & Validate

In [10]:
param_grid_2 = {'alpha':[0.001,0.01,0.05,0.1,0.5,1.0], 'l1_ratio':[0.1,0.5,0.9]}
best2, rmse2, _ = ts_grid_search(X2_tr, y2_tr, param_grid_2, n_folds=N_FOLDS, random_state=SEED)
print('Stage2 best:', best2, '| CV RMSE:', rmse2)
m2 = fit_enet(X2_tr, y2_tr, best2, random_state=SEED)
pred_tr = m2.predict(X2_tr); pred_te = m2.predict(X2_te)
mt_tr = summarize(y2_tr, pred_tr); mt_te = summarize(y2_te, pred_te)
print('Train metrics:', mt_tr)
print('Test metrics:', mt_te)

Stage2 best: {'alpha': 1.0, 'l1_ratio': 0.9} | CV RMSE: 56125.47563025254
Train metrics: {'MAE': 24750.7774484828, 'MAPE': 2150.662410838068, 'RMSE': 32279.81168328084}
Test metrics: {'MAE': 55655.28173193526, 'MAPE': 5167.922003405911, 'RMSE': 63256.68571054362}


## 5) Diagnostics & Artifacts

In [11]:
plot_tr = train2[[DATE_COL, Y_COL]].copy(); plot_tr['yhat']=pred_tr
plot_te = test2[[DATE_COL, Y_COL]].copy();  plot_te['yhat']=pred_te
save_actual_vs_pred(plot_tr, Y_COL, 'yhat', path=str(REPORTS/'train_actual_vs_pred.png'))
save_actual_vs_pred(plot_te,  Y_COL, 'yhat', path=str(REPORTS/'test_actual_vs_pred.png'))
plot_tr['resid'] = plot_tr[Y_COL]-plot_tr['yhat']; plot_te['resid'] = plot_te[Y_COL]-plot_te['yhat']
save_residual_plots(plot_tr['resid'].values, prefix=str(REPORTS/'train_resid'))
save_residual_plots(plot_te['resid'].values, prefix=str(REPORTS/'test_resid'))

# Sensitivity
if 'average_price' in X2_te.columns:
    base = pred_te.copy()
    up = test2.copy(); up['average_price'] *= 1.05
    down = test2.copy(); down['average_price'] *= 0.95
    y_up = m2.predict(up.drop(columns=[DATE_COL, Y_COL], errors='ignore'))
    y_down = m2.predict(down.drop(columns=[DATE_COL, Y_COL], errors='ignore'))
    pd.DataFrame({'base_pred':base,'price_up_5pct':y_up,'price_down_5pct':y_down}).to_csv(REPORTS/'sensitivity_price.csv', index=False)

if 'promotions' in X2_te.columns:
    toggled = test2.copy(); toggled['promotions'] = 1 - toggled['promotions']
    y_tog = m2.predict(toggled.drop(columns=[DATE_COL, Y_COL], errors='ignore'))
    pd.DataFrame({'pred_base':pred_te,'pred_toggle_promo':y_tog}).to_csv(REPORTS/'sensitivity_promo.csv', index=False)

dump(m2, REPORTS/'model.joblib')
print('Artifacts saved to', REPORTS)

Artifacts saved to c:\Users\bimal\Downloads\mmm_mediation_repo\mmm_mediation_repo\reports
